<a href="https://colab.research.google.com/github/ido177/colab/blob/main/select/select_examples.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!gdown --id 1BSHIKQ7rFw5BpTq5nw1UZfjPK_7Mpnbi
!mv _sqlite3.cpython-37m-x86_64-linux-gnu.so /usr/lib/python3.7/lib-dynload/
import os
os.kill(os.getpid(), 9)

Downloading...
From: https://drive.google.com/uc?id=1BSHIKQ7rFw5BpTq5nw1UZfjPK_7Mpnbi
To: /content/_sqlite3.cpython-37m-x86_64-linux-gnu.so
6.33MB [00:00, 99.0MB/s]


# Грузим БД

In [1]:
import pandas as pd
import numpy as np
import sqlite3

In [2]:
sqlite3.sqlite_version

'3.36.0'

In [3]:
con = sqlite3.connect('db')


In [4]:
!gdown --id 1Gj0qVU0Acv-HqRjTIEwvkO4TqJYijehR

Downloading...
From: https://drive.google.com/uc?id=1Gj0qVU0Acv-HqRjTIEwvkO4TqJYijehR
To: /content/german_credit_augmented.csv
100% 69.6k/69.6k [00:00<00:00, 65.0MB/s]


In [5]:
df = pd.read_csv('/content/german_credit_augmented.csv')

In [6]:
df.head()

,age,sex,job,housing,saving_accounts,checking_account,credit_amount,duration,purpose,default,contract_dt,client_id
0,33,male,2,own,NaN,NaN,3074,9,radio/TV,0,2008-06-29 18:52:00,210
1,43,male,1,own,little,little,1344,12,car,0,2007-05-20 18:30:19,929
2,52,male,2,own,quite rich,NaN,936,9,education,0,2008-04-27 08:23:07,200
3,35,female,3,own,little,NaN,1393,11,car,0,2007-05-06 10:58:22,45
4,28,male,2,own,little,NaN,776,12,radio/TV,0,2007-07-21 13:22:14,358


In [7]:
df['contract_dt'] = pd.to_datetime(df['contract_dt'],format='%Y-%m-%d %H:%M:%S')

In [8]:
df.dtypes

age                          int64
sex                         object
job                          int64
housing                     object
saving_accounts             object
checking_account            object
credit_amount                int64
duration                     int64
purpose                     object
default                      int64
contract_dt         datetime64[ns]
client_id                    int64
dtype: object

In [9]:
df.to_sql('german_credit',con,index=False,if_exists='replace')

In [10]:
def select(sql):
  return pd.read_sql(sql,con)

# Селектим

In [11]:
sql = '''SELECT t.age * 3 AS age_mult3, 
    t.housing, 
    rc.housing, 
    us.job 
FROM german_credit AS t
LEFT JOIN russian credit AS rc 
    on ...
LEFT JOIN usa_credit AS uc 
    on ...
'''

Нужно избегать тупорылые псевдонимы, типа t1, t2 - можно запутаться\
as можно ставить по-желанию\
t.age * 3 as age_mult3 - умножаем значение на 3 и задаем новое значение колонки

In [12]:
select(sql)

DatabaseError: ignored

In [ ]:
sql = '''SELECT t.age * 3 AS age_mult3,
    t.age + t.credit_amount AS age_plus_amount,
    t.age * 1.0 / t.credit_amount AS age_div_amount
FROM german_credit AS t'''

умножение на единицу позволяет преобрести нужный формат, просто деление может быть нулем

In [ ]:
select(sql)

In [ ]:
sql = '''SELECT * FROM german_credit t 
WHERE t.contract_dt BETWEEN '2007-01-01' AND '2007-12-31'
--AND (t.purpose = 'car' OR t.purpose IN ('repairs'))

ORDER BY contract_dt desc, credit_amount
LIMIT 5
'''

df.head(5) тот же лимит\
desc - по убывающей, credit будет в данном случае по-возрастанию\
-- коммент в sql

In [ ]:
select(sql)

# Like

In [14]:
t = pd.DataFrame({'purpose': ['машина', 'на машину', 'на покупку машины', 'автомобиль', 'на возвращение 2007'],
              'amount':[1000, 400, 600, 700, 1500]})

In [15]:
t.to_sql('purpose', con, index=False, if_exists='replace')

In [33]:
sql = '''select * from purpose t
where (t.purpose like '%машин%'
or t.purpose like '%авто%')
and t.amount > 500
'''

In [34]:
select(sql)

,purpose,amount
0,машина,1000
1,на покупку машины,600
2,автомобиль,700


# case

In [42]:
sql = '''select count(*) from german_credit t
where t.credit_amount > 1000'''

In [43]:
select(sql)

,count(*)
0,884


In [52]:
sql = ''' select t.credit_amount, 
case when t.credit_amount > 1000
then 1 
else 0
end as greater_then_1000_flag,

iif(t.credit_amount > 1000,1,0) as greater_then_1000_flag_2

from german_credit t'''

case - условие создания новой колонки. пишется в селекте. При кредит эмаунте большем 1000 - значение будет 1\

iif в принципе то же самое. 1ое число - значение, второе тру, третье фэлс

In [53]:
select(sql)

,credit_amount,greater_then_1000_flag,greater_then_1000_flag_2
0,3074,1,1
1,1344,1,1
2,936,0,0
3,1393,1,1
4,776,0,0
...,...,...,...
995,2600,1,1
996,4455,1,1
997,6403,1,1
998,5003,1,1


In [59]:
sql = ''' select avg(case when t.credit_amount > 1000 then 1 else 0 end) as greater_then_1000_flag
FROM german_credit t'''

In [60]:
select(sql)

,greater_then_1000_flag
0,0.884


In [61]:
cur = con.cursor()

In [68]:
sql = '''
drop table if exists greater_then_1000_credit;

create table greater_then_1000_credit as
select * from german_credit t
where t.credit_amount > 1000'''

In [69]:
cur.executescript(sql)

cur.execute(sql) для одного запроса\

cur.executescript(sql) для нескольких\

разделение с помощью ;

!!!Это только sqlite!!!

In [70]:
sql = '''select * from greater_then_1000_credit'''

In [71]:
select(sql)

,age,sex,job,housing,saving_accounts,checking_account,credit_amount,duration,purpose,default,contract_dt,client_id
0,33,male,2,own,None,None,3074,9,radio/TV,0,2008-06-29 18:52:00,210
1,43,male,1,own,little,little,1344,12,car,0,2007-05-20 18:30:19,929
2,35,female,3,own,little,None,1393,11,car,0,2007-05-06 10:58:22,45
3,27,female,2,own,little,moderate,1295,18,furniture/equipment,0,2008-06-18 04:10:05,86
4,26,male,2,own,little,little,4370,42,radio/TV,1,2007-11-29 00:20:44,639
...,...,...,...,...,...,...,...,...,...,...,...,...
879,65,male,2,free,little,little,2600,18,radio/TV,1,2007-12-16 20:17:19,624
880,30,male,3,own,little,moderate,4455,36,business,1,2007-07-12 14:08:58,181
881,33,male,2,own,little,moderate,6403,24,radio/TV,0,2008-04-08 03:24:26,730
882,29,female,2,own,None,None,5003,21,car,1,2007-11-29 15:51:45,557


# Union

In [72]:
jan = pd.DataFrame({'month': ['jan', 'jan'], 'revenue': [1,2]})
feb = pd.DataFrame({'month': ['feb', 'feb'], 'revenue': [1,2]})

In [73]:
jan.to_sql('jan', con, index=False, if_exists='replace')
feb.to_sql('feb', con, index=False, if_exists='replace')

In [80]:
sql = ''' select t.revenue,
'jan' as month
from jan t
union all
select t.revenue,
'mar' as month
from feb t'''

количество колонок должно быть равным\

просто union удаляет дубли, union all не трогает дубли\

union all - безопасно\

'mar' as month подменяет значение

In [81]:
select(sql)

,revenue,month
0,1,jan
1,2,jan
2,1,mar
3,2,mar
